In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load dataset
file_path = "Online Retail-unclean.xlsx"
data = pd.read_excel(file_path, engine='openpyxl')

# Step 1: Identify missing data
# 1a: Columns with missing data
missing_columns = data.isnull().sum()[data.isnull().sum() > 0]
print("Columns with missing data:")
print(missing_columns)

# Visualization: Missing data per column
missing_columns.plot(kind='bar', figsize=(8, 6))
plt.title("Missing Data by Column")
plt.ylabel("Count of Missing Values")
plt.xlabel("Columns")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 1b: Countries with missing data
missing_countries = data[data.isnull().any(axis=1)]['Country'].dropna().unique()
print("\nCountries with missing data:")
print(missing_countries)

# Step 2: Data Cleaning
# Filling missing values for numerical columns with median and categorical with mode
data_cleaned = data.copy()
for col in data_cleaned.columns:
    if data_cleaned[col].dtype in ['float64', 'int64']:
        data_cleaned[col].fillna(data_cleaned[col].median(), inplace=True)
    elif data_cleaned[col].dtype == 'object':
        data_cleaned[col].fillna(data_cleaned[col].mode()[0], inplace=True)

# Save cleaned data
data_cleaned.to_excel("Online-Retail-Clean.xlsx", index=False)

# Visualization: Compare data before and after cleaning
before_cleaning = data.isnull().sum()
after_cleaning = data_cleaned.isnull().sum()

compare_cleaning = pd.DataFrame({
    "Before Cleaning": before_cleaning,
    "After Cleaning": after_cleaning
})
compare_cleaning.plot(kind='bar', figsize=(10, 6))
plt.title("Comparison of Missing Data Before and After Cleaning")
plt.ylabel("Count of Missing Values")
plt.xlabel("Columns")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Step 3: Justification for imputation
# Median and mode are used as explained in the textual part of the response.

# Step 4: Aggregation
data_cleaned['InvoiceDate'] = pd.to_datetime(data_cleaned['InvoiceDate'])
data_cleaned['Month'] = data_cleaned['InvoiceDate'].dt.to_period('M')
aggregated_data = data_cleaned.groupby(['Country', 'Month']).agg({'UnitPrice': 'sum'}).reset_index()

# Save aggregated data
aggregated_data.to_excel("Aggregasi.xlsx", index=False)

# Visualization: Aggregated data
top_countries = aggregated_data.groupby('Country')['UnitPrice'].sum().sort_values(ascending=False).head(10)
top_countries.plot(kind='bar', figsize=(10, 6))
plt.title("Top 10 Countries by Total Unit Price")
plt.ylabel("Total Unit Price")
plt.xlabel("Countries")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Step 5: Sampling
sample_sizes = {}
alphas = [0.03, 0.01]

for alpha in alphas:
    sample_sizes[alpha] = data_cleaned['Country'].value_counts() * alpha
    sample_sizes[alpha] = sample_sizes[alpha].astype(int)

# Create sampling dataframes for alpha values
sampling_results = pd.DataFrame(sample_sizes)
sampling_results.reset_index(inplace=True)
sampling_results.rename(columns={"index": "Country"}, inplace=True)

# Save sampling data
sampling_results.to_excel("Sampling.xlsx", index=False)

# Visualization: Sampling results
sampling_results.set_index("Country").plot(kind='bar', figsize=(12, 6))
plt.title("Sample Sizes by Country for Alpha 0.03 and 0.01")
plt.ylabel("Sample Size")
plt.xlabel("Country")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\nProcessing complete. Files saved:")
print("- Cleaned data: Online-Retail-Clean.xlsx")
print("- Aggregation data: Aggregasi.xlsx")
print("- Sampling data: Sampling.xlsx")